In [57]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0509 06:53:39.198721 140552650831616 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [5]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [60]:
!mkdir bert_new_prepositions

In [6]:
OUTPUT_DIR = "bert_new_prepositions"

In [7]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_new_prepositions'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
# tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: bert_new_prepositions *****


#Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [8]:
from tensorflow import keras
import os
import re

In [9]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

error_type = "Prepositions" #@param ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
error_ratio = "ToFifteen" #@param ["ToFifteen", "AugmentedRatio"]

error_type = translate_dict[error_type]

It's about time we named our model

In [10]:
modelname = "Prepositions_ToFifteen" # @param {type:"string"}

In [11]:
import shutil

filename = error_ratio+"/train/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'.')
trn = pd.read_json(error_type+".json").reset_index(drop=True)
trn["is_error"] = trn["is_error"].astype(int)

filename = error_ratio+"/test/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'./'+error_type+'_test.json')
tst = pd.read_json(error_type+"_test.json").reset_index(drop=True)
tst["is_error"] = tst["is_error"].astype(int)

In [12]:
len(trn)

54054

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [13]:
train = trn
test = tst

In [14]:
train.columns

Index(['context', 'is_error', 'path', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [15]:
DATA_COLUMN = 'context'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [16]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [17]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
5        <bert.run_classifier.InputExample object at 0x...
6        <bert.run_classifier.InputExample object at 0x...
7        <bert.run_classifier.InputExample object at 0x...
8        <bert.run_classifier.InputExample object at 0x...
9        <bert.run_classifier.InputExample object at 0x...
10       <bert.run_classifier.InputExample object at 0x...
11       <bert.run_classifier.InputExample object at 0x...
12       <bert.run_classifier.InputExample object at 0x...
13       <bert.run_classifier.InputExample object at 0x...
14       <bert.run_classifier.InputExample object at 0x...
15       <bert.run_classifier.InputExample object at 0x...
16       <bert.run_classifier.InputExample object at 0x.

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [18]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'MA' and split_tokens[index + 2] == '##S' and split_tokens[index + 3] == '##K' and split_tokens[index + 4] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 06:55:07.528349 140552650831616 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [19]:
tokenizer.tokenize("This here's an example of using the BERT [MASK] tokenizer")

['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 '[MASK]',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [20]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 54054


I0509 06:55:13.942858 140552650831616 tf_logging.py:115] Writing example 0 of 54054


INFO:tensorflow:*** Example ***


I0509 06:55:13.946878 140552650831616 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0509 06:56:07.171310 140552650831616 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0509 06:56:07.173235 140552650831616 tf_logging.py:115] label: 0 (id = 0)


INFO:tensorflow:Writing example 10000 of 13514


I0509 06:56:17.643501 140552650831616 tf_logging.py:115] Writing example 10000 of 13514


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [21]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [22]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [23]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 12.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [24]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)

40540


In [25]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [26]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_save_checkpoints_steps': 500, '_global_id_in_cluster': 0, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_service': None, '_evaluation_master': '', '_save_checkpoints_secs': None, '_eval_distribute': None, '_experimental_distribute': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd45d6ae278>, '_num_worker_replicas': 1, '_protocol': None, '_keep_checkpoint_max': 5, '_is_chief': True, '_task_id': 0, '_num_ps_replicas': 0, '_model_dir': 'bert_new_prepositions', '_tf_random_seed': None, '_train_distribute': None, '_master': '', '_device_fn': None}


I0509 06:56:35.578891 140552650831616 tf_logging.py:115] Using config: {'_save_checkpoints_steps': 500, '_global_id_in_cluster': 0, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_service': None, '_evaluation_master': '', '_save_checkpoints_secs': None, '_eval_distribute': None, '_experimental_distribute': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd45d6ae278>, '_num_worker_replicas': 1, '_protocol': None, '_keep_checkpoint_max': 5, '_is_chief': True, '_task_id': 0, '_num_ps_replicas': 0, '_model_dir': 'bert_new_prepositions', '_tf_random_seed': None, '_train_distribute': None, '_master': '', '_device_fn': None}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [27]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [83]:
print('Beginning Training!')
current_time = datetime.now()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/gpu:0'):
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0508 22:20:57.239850 139883003524864 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0508 22:21:00.934807 139883003524864 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 40540 into bert_new_prepositions/model.ckpt.


I0509 02:39:19.952416 139883003524864 tf_logging.py:115] Saving checkpoints for 40540 into bert_new_prepositions/model.ckpt.


INFO:tensorflow:Loss for final step: 3.043557e-06.


I0509 02:39:24.187605 139883003524864 tf_logging.py:115] Loss for final step: 3.043557e-06.


Training took time  4:18:52.397762


In [1]:
print("God help me.")

God help me.


Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Now let's use our test data to see how well our model did:

In [28]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [30]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0509 06:57:07.697016 140552650831616 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 06:57:11.744238 140552650831616 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 40540: auc = 0.68536246, eval_accuracy = 0.95160574, f1_score = 0.49614787, false_negatives = 523.0, false_positives = 131.0, global_step = 40540, loss = 0.5112707, precision = 0.7108168, recall = 0.3810651, true_negatives = 12538.0, true_positives = 322.0


I0509 06:58:23.480092 140552650831616 tf_logging.py:115] Saving dict for global step 40540: auc = 0.68536246, eval_accuracy = 0.95160574, f1_score = 0.49614787, false_negatives = 523.0, false_positives = 131.0, global_step = 40540, loss = 0.5112707, precision = 0.7108168, recall = 0.3810651, true_negatives = 12538.0, true_positives = 322.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 40540: bert_new_prepositions/model.ckpt-40540


I0509 06:58:26.319608 140552650831616 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 40540: bert_new_prepositions/model.ckpt-40540


{'auc': 0.68536246,
 'eval_accuracy': 0.95160574,
 'f1_score': 0.49614787,
 'false_negatives': 523.0,
 'false_positives': 131.0,
 'global_step': 40540,
 'loss': 0.5112707,
 'precision': 0.7108168,
 'recall': 0.3810651,
 'true_negatives': 12538.0,
 'true_positives': 322.0}

#Extracting the trained model


Now let's save our model:

In [31]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [32]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set[:128])
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="blue">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [34]:
import json

curname = "wsites_" + modelname + ".json"

test_texts = {"./data/exam/exam2014/AMe_8_1.txt": "The graph provides information about the skill levels after school in Australia in 1999 and the correlation of males and females who had them.\n\t As can be seen from the graph the vast majority of skilled vocational diploma accounted for 90% of males, while females not interested that. (a mere 10 per cent). In 1999 undergraduate diploma and postgraduate diploma was 70 per cent of females and males respectively. Whereas the undergraduate diploma accounted for only about 37 per cent for males and postgraduate was only 30 per cent.\n\tFurthermore the Bachelor\u2019s degree was more popular for women (about 55 per cent) than for men was almost half. The percentages of master\u2019s degree for female was 40 per cent in 1999 and the percentage of males to reach 60 per cent.\n\tOverall, the qualification after school in Australia in 1999 took a big part by males, than females.\n", "./data/exam/exam2014/2012-2014_4/esl_00945.txt": "If tomorrow comes by Sidney Sheldon\nReview by Demkina Daria\n\"If tomorrow comes\" was written in 1985 and its genre is crime fiction. This book narrates us a story of a woman who has become the victim of New Orlean Mafia's criminal acts and who has properly conceived a plan of revenge. \nThe narration is third-person, on behalf of the author. The book is set in the time of its writing, actions unfold all over the world, beginning in New Orlean.\nThe main character is Tracy Whitney, an ordinary woman, who had exemplary fiance, lucrative job and settled life until her mother got in the way of serious Mafia's members in New Orlean in consequence of which she died. From that moment the life of Tracy had changed cardinally. Trying to revenge for her mother she was totally defeated by the top of criminal gang ruling in New Orlean: Joe Romano, Judge Lawrence, Orsatti and Perry Pope. This crushed woman lost everything she had, even an unborn child. She was sentenced to fifteen years in jail for a crime that she didn't commit. Being in prison she vowed herself to revenge to all those people who were related to her miseries. All in all she got free ahead of time and set about the plan's realization. Tracy Whitney had destroyed all her enemies and began defeat the most authoritative and prosperous members of the Mafia plunging into adventure, making acquaintances and robbing all of them in the great amount of money. Soon she became obsessed with such criminal tricks and could not stop anymore until she had met another smart robber as she was, Jeff Stevens, and fall in love with him.\nI absolutely love this book and would recommend it to everyone.  It is a real page turner that can capture everyone by its gripping thought-provoking plot and moving characters. First of all it is an educative story that shows us how not to give up even in the most difficult and leading into deadlock situations. And how important to stay calm and rational in such extreme conditions to survive. On the other side we see how dangerous to plunge into such adventures, how important to stop in time and not become obsessed with revenge. \nThe main idea of the book is following: all people's evil exploits will certainly be sooner or later repaid as it happened with those, who had made money illegally, destroying the lives of innocent people.\nThroughout the narration we face to great characters that strike us by their incredible dispositions.\nAnd after reading this fascinating book I decided to read others books of Sidney Sheldon.\n", "./data/exam/exam2014/EPa_5_1.txt": "The bie charts illustrate the persentage of ages of people in Yemen and Italy in 2000 and 2050. It is clearly seen that the biggest population are and will be people at 15-59 ages.\nOne of the most obvious fact in charts that in Yemen in 2050 there will be less children in 0-14 ages, than it was in 2000. There will be sharply decline from 50,1% to 37,0%. Furthermore, there is projection the persentage of older people at 60 and more years, will stay the same. As for Italy, it is seen that there is trend that persentage of older people will increase dramatically, almost 2 times greater than in 2000. There will be the peak at 42,3%.\nComparing two countries, it is clearly seen that in 2000 there was different persentage of 0-14 years children. In Italy there was 14,3% that is almost 4 times less than in Yemen. But as for people at 15-59, there was almost the same persentage. In Italy there was 61,6%, which there is a bit more than 46,3% in Yemen.\nTo sum up, in general case the ages of the population of Yemen and also of Italy in 2000 and 2050 will be the same.\n", "./data/exam/exam2014/TSha_2_1.txt": "The linear graph demonstrates the development of number of senior citizens in three countries throughout a century. There are some interesting facts concerning the population of people aged 65 and over in Japan, Sweden and the USA in the period between 1940 and 2040.\nThe overall trend in each country is upward, though there is one exception to this which occured in Japan and lasted from 1940 till the beginning of 21st century. After this modest deep the proportion of retirees roared dramatically and constituded over a quarter at the end of the period.\nIt is obvious that in the beginning of the period the fraction of senior citizens was less then 10%, with the USA having the largest share and Japan \u2013 the least.\nIn the USA the number of people aged 65 and over fluctuated slightly and reached a plateau in the first 20 years of 21st century. Afterwards, there was a moderate increase with maximume of just under a quarter.\nSimilar development may be observed in the proportion of senior citizens of Sweden. It is also worth pointing out that the number of this age group fluctuated throughout the whole period and reached a peak at approximately a quarter of the country\u2019s population.\n", "./data/exam/exam2014/2012-2014_2/esl_00380.txt": "Transport has already taken a great part of our living and in future its influence is only going to increase. Owning to fast pace of life most people spend great amount of their time on transportation. It might be one of the main reasons why some people are concerned with the purposes and usage of transport in future. In my essay I would like to share some of my considerations about the possible and desired changes in this sphere of humans' life. \nTo begin with I would like to argue that transport in future should be more environmentally friendly. Almost all the cars today are drawing petrol or diesel fuel and produce carbon dioxide and other harmful substances to the atmosphere. Moreover, oil from which petrol and diesel fuel are obtained is a non-renewable recourse and soon it will certainly run out. It's common knowledge that new types of electric vehicle had already been invented, although they are too expensive and as a result not spread enough. \nAnother big problem today is permanent traffic congestions in every big city. This issue can be solved in many different ways but I'm fancy of suggesting the idea of creating such means of transport that won't require a track to ride. As far as I remember one of the science-fiction writer presenting the idea of creating cars that will move due to the terrestrial magnetic field. It might settle simultaneously the problem of air pollution and traffic jams. \nAs we are living in the age of information technologies speed of movement has a great significance. I'm convinced that transport in future will be able to transport people as fast as information, for instance e-mails of messages in Skype, now. \nNobody knows what may happen tomorrow, in two weeks, in couple of years, we can predict, though, and we are vigorous enough to influence on our future and make it better. It refers as well to all spheres of our life and to every person in particular.\n", "./data/2012-2014/esl_00330.txt": "How to solve the problem of traffic jams in Moscow?\nFrom year to year the transport situation in Moscow is getting worse. The main problem is obviously the problem of traffic congestions. Considerable social and financial resources are sent (are put? I'm not sure what verb I should use here: \u043d\u0430\u043f\u0440a\u0432\u043b\u0435\u043d\u044b) on solving this problem. I would like to make some proposals which may make our transport better.\nFirst of all, we know that lack of parking spaces is a real headache for drivers. That's why there should be intercept parkings close to subway stations, which are located near Ring Road (\u044f \u0438\u043c\u0435\u044e \u0432 \u0432\u0438\u0434\u0443 \u041c\u041a\u0410\u0414, \u043d\u0435 \u0443\u0432\u0435\u0440\u0435\u043d\u0430, \u0442\u0430\u043a \u043b\u0438 \u043e\u043d \u043f\u0435\u0440\u0435\u0432\u043e\u0434\u0438\u0442\u0441\u044f \u043d\u0430 \u0430\u043d\u0433\u043b\u0438\u0439\u0441\u043a\u0438\u0439). For instance, near such stations as Domodedovo, Teply stan, where there is a lot of space. The other thing we have to mention is that there are a great large of lawns in Moscow. They occupy large spaces, which surely could be used as parkings. Also, at the expense of these lawns we could widen our roads. (I'm not talking about trees, just lawns). Because of the fact that Moscow is a capital, there is a great deal of government institutions of federal and regional subordination. As a consequence, lots of people go to the centre every morning. This is another cause of traffic congestion (not ordinary people but those with flashing lights on the roof of the car). The possible solution can be moving these institutions to the suburbs. However, before doing it we should pave the road to avoid traffic jams there. Furthermore, we can pave several chords in Moscow, because now the whole traffic is based on the principle of rings and radii.\nIn conclusion I would like to add that there is no doubt that we can find many more solutions to the problem. But the main thing is just to start doing something, because here, in Moscow this problem has come to unthinkable extent.\n", "./data/old IELTS/IELTS2016/JSl_95_1.txt": "The bar chart below reports the amount of financial supply into renewable energy in period from 2006 to 2013.it can be seen that investments are made by developed and developing countries and the investment of the whole world as well and that over tendency is to rise up the amount of supply. The supply of developed countries had fluctuated between almost 70 bn dollars and 100 bn dollars in the period from 2006 to 2009 while the investment of developing countries had permanently move up to 60 bn in 2009 from 2010 to 2013 the developed countries investment had hit the highest point in 2011 and moderately reduce in 2013. Meanwhile developing countries had go up to 100 bn in 2011 and decline in 2013 had slowly grow to 171 bn in 2008. After a short stagnation it had jumped up to 279 bn in 2011. Afterward it plummeted to 214 bn in 2013. \n", "./data/old IELTS/IELTS2015/ADe_18_1.txt": "This table indicates the number of people in different age groups working in sectors in UK such as building, education and other.\nFirst of all, we can see that people aged 18-25 years, 26-35 years and 36-45 years were most popular specialists by sectore of technology. For example, there were 454375 workers in 2006. Also in comparison with 1998 number of people from all groups were dramatically risen. We can see the same tendention by sector of education. \nIn contrast to these sectors the number of people from all groups left the sector of building. There were 112565 specialists in age group 18-25 years old in 1998 year and it has fallen to 86430 workers. The same tendention was in all age groups.\nTo summ up it all, we can say that people from all age groups prefer to go and work in sectors such as education and technology because the number of people by these sectors grew up.\n", "./data/2012-2014/esl_00107.txt": "Nowadays there are a lot of different magazines newspapers and others sources of various information from politics to culture. All pieces of news appear due to the work of journalists. Today there are a lot of opinions about the jobs in journalism: is it a good profession to make a living or not?\nFrom my point of view, journalism is a good profession. First of all, it is a very important kind of job for all people. Due to journalists we get information from all over the world. They make it accessible through the Internet,  TV,  magazines or newspapers, or  radio. They sometimes have to go very far away from their home to make a report from some places where something interesting or even dangerous is happening.  Just imagine how little we would know about what is happening around if there were no journalists. It also should be mentioned that due to rating wars journalists from different media try to never miss a deadline. That is why we also get the latest news in a timely manner. Secondly, journalism is a very interesting profession. Working as a correspondent you often have to go to very unusual places, sometimes even to dangerous areas. Using the skills of getting the required information, you make a good report for people and in addition you learn a lot of new things, see new places, meet people and, what is more important, improve journalistic skills. Journalists should provide the quality material. That is why they have to understand theater, art, politics, which makes their life bright and rich. Furthermore, journalism is a useful profession for the person who works in this sphere. People wanting to become a good journalist in getting their higher education degree learn how to write clear and concise sentences, how to process the required information. Moreover, it is important to be able to deal with different people. So being a journalist you develop personal qualities such as trust, patience, perseverance, tact and tolerance, and communicative skills. All these qualities help people to be successful at work. For example,  author and presenter of the TV program \"Top Secret\" Stanislav Kucher demonstrates \"creative hearing\". Free, quiet manner of conversation, respect, unbiased attitude allow the journalist to discuss even the most sensitive issues without using manipulative techniques.\nHowever, some people are confident that journalism is a \"dirty\" job because they often make money on other people's mistakes or misfortunes. People suppose that journalist will do almost everything for a scoop or to break a story. I do not agree with this statement. If you are a well educated person you know all libel laws and follow them. Moreover, respectable press companies do everything to maintain their reputation. So they keep an open mind and always check the sources of information before providing a story, and they always respect the privacy of public people. What is  important  there are companies whose work is aimed at monitoring  media. For example, PR News is a specialized agency to analyze PR-campaigns and media monitoring in Moscow and other regions of Russia and Western Europe. The company uses two ways of tracking press services. The first one is the audit of the press service for the period of time. The second method is based on feedback from journalists and other media: editors, major bloggers and so on. Skilled, precise, correct survey of journalist's work helps diagnose the problem and prevent its negative effects. Two described methods help determine how precise and objective information of a certain press-service .\nTo summarize, there are some arguments about the profession of journalist. But there is no doubt that this vocation brings a lot of good to people from all over the world as well as to journalists themselves in a way of developing a lot of human skills.\n", "./data/exam/exam2017/EGe_220_1.txt": "The pay chart give information about the transportation of goods between countries in Eastern Europe.\nAs it is evidence from the chart there were two types goods transported: by rail and by road. The largest number of goods transported by rail were metals. The number of these goods were 35 percent.The lowest number were non-metallic minerals. Unfortunatelly there were not information about number of percent.Chemicals and food product transported like machinery. It is about 11-12 percent.\nThe transportation of goods by road shows that the food products jumped. The number of percent are 30. But machinery are far less from transporte by rail, roughly 2 percent.Non-metallic minerals are the same of metals about 11-13 percent.\nIn conclusion the transportation of goods in Eastern Europe in 2008 was very good. Metals were the best transported by rail and food products people from Europe gave by road. The best way of manufactured goods were by road. Machinery were not popularity in Eastern Europe in 2008. The number decrease by rail and road.", "./data/exam/exam2016/JSl_140_2.txt": "In the modern world plenty of international companies believe that it is the good idea to change location of their businesses. So poorer developing countries with their local workers become the most brilliant place for it. First of all, if companies employ local workers for their businesses, they will economy a lot of money on salaries. In developing countries the average income is noticeable lower than in developed countries. Secondly, there are not a lot companies in developing countries. So there is a terrific place for new ideas and start-ups. Moreover, it is quite unexpensive to move your company to such country. For example, pays for place, which company orders, will be much cheaper like an other different pays. On the other hand, such businesses may have some disadvantages too. Firstly, there is no guaranty that company will reach a success. For example, if the company choose the wrong country for their business  they will not find their consumers and then they will lose their money. Lso the economy in developing countries is not safe enough for expensive businesses. We can see the example by situation in breece. They have a big problems with money so some businesses, which was moved there probably had to close. In conclusion, the advantages can outweigh the disadvantages. But it depends in some factors such as country economy, interests of consumers, type of business, etc. this companies help particular countries to develop their economy. \n", "./data/exam/exam2014/MTsy_34_2.txt": "In our world there are a lot of global problems and one of them is the worse public health. Someone believes that a good decision of this will be an increase of the amount of sports facilities, but others have another point of view.\nAnyway, a rise of sport centres, swimming pools, gems, etc. is a good way of solving this problem. First of all, it means that sport facilities will be near people's houses and consequantly they will not make spend too much time on just a road. Moreover, the huge number of fitness centres can impact on people's mind and help them to understand the importance of doing exercises.\nNevertheless, an easy access to sport facilities can not be the best variant in improving public health. Some people simply have not any desire to do sport and do not want to think about how it is useful. Besides, nowadays adults and expesially youngsters prefer spent their time playing computer games or chatting with friends in social networks and are not interesting in their helth.\nAs for me, I am not strongly sure that the vast amount of sports facilities may ensure the improving of population health. In my opinion the first step must be a significant drop in production of junk food, because it is a reason of bad health in the last decades in almost all countries in the world.\nTo sum up, I want to stress that the best way of improving health is a produsing less harmful food and creating better environmental facilities, not sports ones.\n", "./data/old IELTS/IELTS2015/EEm_24_2.txt": "It is strongly believed that some subject in universities are hard explain and understand for female students and there should be more boys to study specific areas of science. However, some people believe that women can benefit to various subjects in many ways.\n\nOn the one hand, the number of male students is very important for universities. First of all, male concentrate on subjects, whereas women distract boys and themselves on feelings and may do not pay appropriate attention to the process of studying. In addition, a man should build his life and find an excellent job in order to accomplish independence and confidence, and education is the main key to starting up a self-made project and following dreams.\n\nOn the other hand, female students should accepted to every subject at university. To start with, they are good business-women and more responsible for their job than man. It means that a well-quallified female worker in the different field of studies is welcomed. Besides, women have different point of views on specific subjects and sometimes with the various outlooks they help to make a scientific discovery.\n\nAlthough some subjects may seem difficult for understanding for women, I strongly believe that the number of them should be equal with the male students. It makes the pocess of studying more interesting. More than that, women may find some interesting details in the research which have been neglected by men. All in all, despite being difficult for studying female students may give a fresh look to the scients, which is why the should have similar right with male students to study different subjects.\n", "./data/exam/exam2014/EPa_6_1.txt": "The pie charts represent information about the ages of the population of Yemen and Italy in 2000 and preductions for 2050.\nIn Yemen in 2000 more than half of population was 0-14 years in comparison with Italy where more than half of population was 15-59 years. The smallest proportion in Yemen was 60+ years, only 3,6%. In Italy the smallest proportion was 0-14 years. In Yemen there will be substantial decrease in children population. In Italy in 2050 will be sharp growth of 60+ years population from 24,1% to 42,3%. Compare with Yemen where growth will be from 3,6% to 5,7%. Yemen`s 15-59 years population will increase to 57,3% compare with Italy, where this population will have big declination from 61,6% to 46,2%.\nTo sum up, in 2000 poputation in Yemen was elder than in Italy. In 2050 population in Italy will be older, because proportion of 60+ years rise and proportion of 15-59 years population decrease.\n", "./data/exam/exam2017_6/OBy_103_2.txt": "It is clear that musicians and film producers lose a gradually value of their money from illegal pirate copies every year. But often people can\u2019t buy a disk, because they have a little money, and disks can crash after some times of using. So I think that it\u2019s nor very well, but it\u2019s true a way for this people. And I agree with them.\n\nBut! People downloads copies of films in their computers, but the can often go to the cinema, because there are many films look like so good and cool on the big (\u044d\u043a\u0440\u0430\u043d?). And often a lot of money comes to the cassa, and they a piese of this money go to the authors of this films. And people often buy disks, if they have special.\n"}
outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 128


I0509 06:59:52.096188 140552650831616 tf_logging.py:115] Writing example 0 of 128


INFO:tensorflow:*** Example ***


I0509 06:59:52.099165 140552650831616 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0509 07:01:40.103366 140552650831616 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0509 07:01:40.165560 140552650831616 tf_logging.py:115] Done running local_init_op.




Annotation took time  0:00:06.943692


In [53]:
os.listdir(".")

['.ipython',
 '.cache',
 '.ipynb_checkpoints',
 'bert_prepositions',
 'delete.json',
 '.jupyter',
 '.bashrc-anaconda3.bak',
 '.config',
 'REALEC_AutoAnnotator_BERT_pipeline Often_confused.ipynb',
 'Prepositions_test.json',
 'wsites_OftenConfused_ToFifteen.json',
 'Often_confused_test.json',
 'Often_confused.json',
 'anaconda3',
 'Anaconda3-4.0.0-Linux-x86_64.sh',
 'download_google_drive',
 '.nano',
 '.bash_history',
 'wsites_Prepositions_ToFifteen.json',
 'Capitalisation.json',
 'Prepositions.json',
 'gdrive-linux-x64',
 '.keras',
 'delete_test.json',
 '.python_history',
 'nltk_data',
 'wsites_delete_ToFifteen.json',
 'wsites_Capitalisation_ToFifteen.json',
 'Datasets',
 '.local',
 'wsites_lex_item_choice_AugmentedRatio.json',
 'REALEC_AutoAnnotator_BERT_pipeline Capitalisation.ipynb',
 'jupyter-drive',
 'cuda-repo-ubuntu1604_9.0.176-1_amd64.deb',
 '.continuum',
 '.conda',
 '.bashrc',
 '.nv',
 'REALEC_AutoAnnotator_BERT_pipeline Prepositions.ipynb',
 'bert_output',
 'Capitalisation_tes

In [54]:
outie[4]

'<html>\n<head>\n<title>./data/exam/exam2014/MTsy_34_2.txt</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>In our world there are a lot of global problems and one of them is the worse public health. Someone believes that a good decision of this will be an increase of the amount of sports facilities, but others have another point of view. Anyway, a rise of sport centres, swimming pools, gems, etc. is a good way of solving this problem. First of all, it means that sport facilities will be near people\'s houses and consequantly they will not make spend too much time on just a road. Moreover, the huge number of fitness centres can impact on people\'s mind and help them to understand the importance of doing exercises. Nevertheless, an easy access to sport facilities can not be the best variant in improving public health. Some people simply have not any desire to do sport and do not want to think

In [35]:
!./gdrive-linux-x64 upload wsites_Prepositions_ToFifteen.json

Uploading wsites_Prepositions_ToFifteen.json
Uploaded 1JKEI0vABre_Rj3wd6dCAT7GkSpZtWItN at 18.8 KB/s, total 27.0 KB


In [41]:
!ls -lAh ./bert_prepositions

total 7.2G
-rw-rw-r-- 1 ivantorubarov ivantorubarov  274 May  8 20:31 checkpoint
drwxr-xr-x 2 ivantorubarov ivantorubarov 4.0K May  8 20:38 eval
-rw-rw-r-- 1 ivantorubarov ivantorubarov 339M May  8 20:31 events.out.tfevents.1557343592.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov 322M May  8 19:27 graph.pbtxt
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  8 20:30 model.ckpt-10000.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  8 20:30 model.ckpt-10000.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  86M May  8 20:30 model.ckpt-10000.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  8 20:31 model.ckpt-10134.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  8 20:31 model.ckpt-10134.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  86M May  8 20:31 model.ckpt-10134.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  8 20:21 model.ckpt-8500.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  8 20:21 model.ckpt-

In [ ]:
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.data-00000-of-00001
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.index
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.meta

Uploading ./bert_output/model.ckpt-2396.data-00000-of-00001
234.9 MB/1.3 GB, Rate: 41.1 MB/s                  

In [1]:
!rm -rf ./bert_output

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 16_pmUKOHS5lKXM2o-1QyVcPQohalCRZd


# Preparing existing model for REALEC production